In [4]:
from data.samples import get_goal_driven_examples
from llm_explain.models.diff import explain_diff, KSparseRegression
import numpy as np

### Comparing datasets of airline reviews.

In this dataset, the dominant difference is about food, but the service attitude is also a big difference. We will show that fitting a linear model on top can help us extract multiple explanations.

In [2]:
dataset = get_goal_driven_examples()

If we naively apply our method, all top explanations might have similar meaning.

In [3]:
result = explain_diff(**dataset, proposer_num_rounds=5, proposer_num_explanations_per_round=3, proposer_precise=True)

print(result)

Printing top 3 explanations:
Explanation: expresses satisfaction with in-flight dining experience; specifically, the text conveys enjoyment or delight with the meal service provided. For example, 'vraiment satisfait de mon expérience culinaire.'
Accuracy: 0.7857142857142857

Explanation: describes the quality of food; specifically, the text makes reference to the enjoyment or taste of meals provided. For example, 'le dîner est excellent'
Accuracy: 0.7857142857142857

Explanation: focuses on culinary experiences; specifically, the text highlights different dishes or flavors offered during the flight. For example, 'meilleur repas que j'ai eu depuis longtemps'
Accuracy: 0.75




In [5]:
### Extracting multiple explanations by fitting a linear model on top

feature_vals = np.array(result.validation_results).T

explanation_coefs = KSparseRegression(feature_vals, result.Y, K=2)

for explanation_idx, explanation in enumerate(result.all_explanations):
    if explanation_coefs[explanation_idx] != 0:
        print(f"{explanation_idx}: {explanation}")
        print(f"  coef: {explanation_coefs[explanation_idx]}")

2: expresses satisfaction with in-flight dining experience; specifically, the text conveys enjoyment or delight with the meal service provided. For example, 'vraiment satisfait de mon expérience culinaire.'
  coef: 2.734332463950064
4: praises the attentiveness of the cabin crew; specifically, the text describes the cabin crew's efforts to provide exceptional service. For example, 'flight attendants went above and beyond.'
  coef: 1.8870186454640325
